In [1]:
from timeit import default_timer
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import corner
import copy
import pandas as pd
%matplotlib inline

import logging

from bilby.core.prior import PriorDict, PowerLaw, Uniform, Sine, Cosine, DeltaFunction
from bilby.gw.conversion import bilby_to_lalsimulation_spins,  generate_component_spins, generate_component_masses
from pycbc.detector import Detector
from sifce import datatools, utils, population


logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

pd.set_option('display.max_columns', None)

In [2]:
prior_waves=dict(mass_ratio=PowerLaw(alpha=0, name='mass_ratio', minimum=0.125, maximum=1),
                mass_1= PowerLaw(alpha=-1, name='mass_1', minimum=10, maximum=80),
                a_1 = Uniform(name='a_1', minimum=0, maximum=0.99),
                a_2 = Uniform(name='a_2', minimum=0, maximum=0.99),
                tilt_1 = Sine(name='tilt_1'),
                tilt_2 = Sine(name='tilt_2'),
                phi_12 = Uniform(name='phi_12', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                phi_jl = Uniform(name='phi_jl', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                luminosity_distance = PowerLaw(alpha=2, name='luminosity_distance', minimum=50, maximum=2000, unit='Mpc', latex_label='$d_L$'),
                dec =  Cosine(name='dec'),
                ra =  Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                theta_jn =  Sine(name='theta_jn'),
                psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),
                phase =  Uniform(name='phase', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                reference_frequency = DeltaFunction(20),
                )
PDict = PriorDict(dictionary=prior_waves)
samples = PDict.sample(20)
samples = generate_component_masses(samples)
samples = generate_component_spins(samples)

In [3]:
samples_df = pd.DataFrame(samples)

waveform_generation_labels = [
    "mass_1",
    "mass_2",
    "spin_1x",
    "spin_1y",
    "spin_1z",
    "spin_2x",
    "spin_2y",
    "spin_2z",
    "theta_jn",
    "phase",
]
detector_position_labels = [
    "ra",
    "dec",
    "psi",
]


delta_f = 1 / 8.0
f_start = 20
f_max = 512
approximant = "IMRPhenomXPHM"
end_time = 10000000


detectors = dict()
psds = dict()
for ifo in ['H1', 'L1', 'V1']:
    psds[ifo] = utils.read_psd_from_txt(
        f"o3_{ifo.lower()}.txt",
        f_min=f_start,
        f_max=f_max,
        delta_f=delta_f,
        basedir="/home/rudall/Projects/SIFCE",
        asd=True,
    )
    detectors[ifo] = Detector(ifo)

snr_sky = []
snr_standard = []
    
samples_df.drop(columns=detector_position_labels) 
    
start = default_timer()
    
for idx, row in samples_df.iterrows():
    wf_params = row[waveform_generation_labels].to_dict()
    hp, hc = datatools.compute_hphc_fd(wf_params, approximant, delta_f=delta_f, f_start=f_start, f_max=f_max)
    
    sky_positions = PDict.sample_subset(keys=detector_position_labels,size=1)
    scattered_snr_opt, _ = datatools.sky_scatter_snr(
        sky_positions,
        dict(
            plus_template = hp,
            cross_template = hc,
            ),
        end_time=end_time,
        psd_dict=psds,
        detector_obj_dict=detectors,
        fmin=f_start,
        fmax=f_max
    )
    for key in psds.keys():
        scattered_snr_opt[key+"_sky"] = scattered_snr_opt.pop(key)[0]
    scattered_snr_opt['net_sky'] = scattered_snr_opt.pop('net')[0]
    snr_sky += [scattered_snr_opt]
    
    strains_dict = dict()
    for ifo in detectors.keys():
        strain = datatools.project_and_combine(
            sky_positions,
            dict(
                plus=hp,
                cross=hc,
            ),
            end_time,
            detector_obj=detectors[ifo],
            
        )
        strains_dict[ifo] = dict(template=strain)
        
    snr_opt, _ = datatools.compute_snr_fd(
        strains_dict,
        psds,
        fmin=f_start,
        fmax=f_max
    )
    snr_standard += [snr_opt]
    
end = default_timer()
print(end-start)

sky_snrs_df = pd.DataFrame(snr_sky)
standard_snrs_df = pd.DataFrame(snr_standard)

samples_df = samples_df.join(sky_snrs_df)
samples_df = samples_df.join(standard_snrs_df)

print(samples_df.keys())

DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [0.87610415],[-0.21574117]
DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'plus_template')
                plus power:2
                 cross_power:0

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'cross_template')
                plus power:1
                 cross_power:1

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('cross_template', 'cross_template')
                plus power:0
                 cross_power:2

DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [-0.71100049],[0.37186558]
DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'plus_template')
                plus power:2
                 cross_power:0

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'cross_template')
                plus power:1
                 cross_power:1

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('cross_template', 'cross_

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('cross_template', 'cross_template')
                plus power:0
                 cross_power:2

DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [0.28233564],[0.55551443]
DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'plus_template')
                plus power:2
                 cross_power:0

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'cross_template')
                plus power:1
                 cross_power:1

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('cross_template', 'cross_template')
                plus power:0
                 cross_power:2

DEBUG:sifce.datatools:project_and_combine: fp, fc are [0.02637539], [-0.82074334]
DEBUG:sifce.datatools:project_and_combine: fp, fc are [-0.18123271], [0.64227924]
DEBUG:sifce.datatools:project_and_combine: fp, fc are [0.28233564], [0.55551443]
DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [0.06462782],[-0

DEBUG:sifce.datatools:project_and_combine: fp, fc are [-0.36418482], [-0.91577566]
DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [0.6980957],[0.45828665]
DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'plus_template')
                plus power:2
                 cross_power:0

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'cross_template')
                plus power:1
                 cross_power:1

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('cross_template', 'cross_template')
                plus power:0
                 cross_power:2

DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [-0.5105539],[-0.58065583]
DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'plus_template')
                plus power:2
                 cross_power:0

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'cross_template')
                plus power:1
                 cross_power:1

D

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'cross_template')
                plus power:1
                 cross_power:1

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('cross_template', 'cross_template')
                plus power:0
                 cross_power:2

DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [-0.58890496],[0.61342314]
DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'plus_template')
                plus power:2
                 cross_power:0

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'cross_template')
                plus power:1
                 cross_power:1

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('cross_template', 'cross_template')
                plus power:0
                 cross_power:2

DEBUG:sifce.datatools:project_and_combine: fp, fc are [0.2723524], [0.37055497]
DEBUG:sifce.datatools:project_and_combine: fp, fc are [0.12106595], [-0.54761

DEBUG:sifce.datatools:project_and_combine: fp, fc are [-0.47568354], [-0.52357615]
DEBUG:sifce.datatools:project_and_combine: fp, fc are [0.11170117], [0.56306186]
DEBUG:sifce.datatools:project_and_combine: fp, fc are [0.19983563], [-0.58000803]
DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [-0.12329608],[0.1952938]
DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'plus_template')
                plus power:2
                 cross_power:0

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'cross_template')
                plus power:1
                 cross_power:1

DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('cross_template', 'cross_template')
                plus power:0
                 cross_power:2

DEBUG:sifce.datatools:sky_scatter_snr: fp, fc are [0.51867088],[-0.18183836]
DEBUG:sifce.datatools:sky_scatter_snr:inner product key ('plus_template', 'plus_template')
                plus power:2
                 cross

1.0812702940002055
Index(['mass_ratio', 'mass_1', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12',
       'phi_jl', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi',
       'phase', 'reference_frequency', 'mass_2', 'iota', 'spin_1x', 'spin_1y',
       'spin_1z', 'spin_2x', 'spin_2y', 'spin_2z', 'phi_1', 'phi_2', 'H1_sky',
       'L1_sky', 'V1_sky', 'net_sky', 'H1', 'L1', 'V1', 'net'],
      dtype='object')


In [4]:
samples_df

,mass_ratio,mass_1,a_1,a_2,tilt_1,tilt_2,phi_12,phi_jl,luminosity_distance,dec,ra,theta_jn,psi,phase,reference_frequency,mass_2,iota,spin_1x,spin_1y,spin_1z,spin_2x,spin_2y,spin_2z,phi_1,phi_2,H1_sky,L1_sky,V1_sky,net_sky,H1,L1,V1,net
0,0.161872,32.337039,0.461524,0.073747,1.389259,2.409976,4.459776,5.300112,1727.098060,-0.529042,5.099198,1.649876,1.641175,4.167378,20.0,5.234461,2.048299,-0.156206,-0.426217,0.083324,-0.040554,0.027977,-0.054875,4.361097,2.537688,3967.990786,4651.631553,674.805835,6151.259213,3967.990814,4651.631513,674.805951,6151.259213
1,0.594491,34.640767,0.807067,0.258517,1.092325,0.643425,3.046044,2.221907,1935.847300,0.377911,6.206141,2.140187,1.799896,3.629664,20.0,20.593612,1.962196,-0.063708,0.713595,0.371592,-0.001009,-0.155091,0.206825,1.659837,4.705882,10161.535571,10574.021309,6019.736065,15852.569322,10161.535581,10574.021293,6019.736105,15852.569332
2,0.465032,44.676593,0.066259,0.957998,2.022930,1.753207,0.043599,0.656763,397.363420,0.254082,3.003381,2.105747,0.209953,1.313784,20.0,20.776061,2.015278,-0.048094,0.035203,-0.028948,-0.783748,0.522780,-0.173782,2.509742,2.553342,7043.983465,11336.242153,2111.509513,13512.459496,7043.983503,11336.242120,2111.509524,13512.459490
3,0.980489,46.984608,0.437650,0.722613,2.523513,1.744187,5.843757,0.144824,525.042880,-0.482647,0.289915,0.679264,1.683685,0.528277,20.0,46.067904,0.687810,-0.232519,0.101246,-0.356681,-0.469707,0.534793,-0.124667,2.730919,2.291491,18232.694249,17073.415785,2282.442306,25082.707370,18232.694231,17073.415801,2282.442278,25082.707366
4,0.658860,10.463179,0.293291,0.257412,0.297208,0.819270,4.015823,4.839709,1900.957270,0.189593,3.376191,1.810027,0.908559,4.875764,20.0,6.893765,1.826423,-0.043186,-0.074244,0.280433,-0.064030,0.176843,0.175749,4.185547,1.918186,1433.665943,1701.497401,707.221899,2334.663628,1433.666229,1701.497213,707.221507,2334.663547
5,0.490510,13.189713,0.925267,0.225545,0.991203,1.925780,5.603173,5.340329,1583.925704,-0.214201,2.652476,0.747178,2.082494,3.290940,20.0,6.469685,0.950712,0.303386,-0.712234,0.506753,-0.057901,-0.203402,-0.078394,5.115076,4.435064,6844.352411,7295.056945,777.197765,10033.297171,6844.352483,7295.056889,777.197666,10033.297172
6,0.925446,14.295186,0.624692,0.033070,1.088468,1.361647,1.898038,4.703095,1736.097896,-0.613954,6.134864,2.549962,2.654061,0.359416,20.0,13.229426,2.656064,0.224991,0.505627,0.289759,-0.032214,0.002953,0.006866,1.152130,3.050168,4558.984762,7342.433068,1820.902273,8832.403439,4558.984806,7342.433044,1820.902363,8832.403460
7,0.848129,13.912963,0.765057,0.535359,1.003959,2.639502,3.907939,4.110611,1877.140776,0.713347,6.237430,0.774146,0.504341,1.944737,20.0,11.799988,0.870578,0.505111,-0.401758,0.410810,-0.256499,-0.024292,-0.469284,5.611264,3.236018,2146.954438,6755.425798,2002.314440,7365.762295,2146.954453,6755.425802,2002.314396,7365.762291
8,0.144675,18.795246,0.597114,0.568104,1.761225,1.994385,5.997127,3.971442,1746.916887,-0.344919,0.048647,0.452846,2.771943,4.202297,20.0,2.719207,1.081021,-0.577471,-0.101485,-0.113022,-0.514644,0.057932,-0.233510,3.315556,3.029498,1108.789554,1119.830327,1703.623389,2320.725509,1108.789576,1119.830239,1703.623658,2320.725674
9,0.542861,61.148535,0.697250,0.413344,1.768437,2.963223,4.501405,2.129410,1983.243993,0.557403,3.756189,1.238503,0.760937,3.855484,20.0,33.195127,0.936473,0.007840,0.683632,-0.136910,0.071531,-0.016180,-0.406786,1.559328,6.060733,11287.576147,13060.836584,3481.127308,17610.027680,11287.576127,13060.836606,3481.127256,17610.027673
